In [4]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import sklearn 
import seaborn as sns


writer_r = pd.ExcelWriter(r'./GBDT.xlsx')

writer_t = pd.ExcelWriter(r'./GBDT_Feature_Ranking.xlsx')

In [ ]:
import pickle
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error,make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import make_scorer, r2_score, mean_squared_error

data_path = r'./data.xlsx'
sheet_names= pd.ExcelFile(data_path).sheet_names
name = 'GBDT'

# Define RMSE
def rmse_scorer(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))
rmse = make_scorer(rmse_scorer, greater_is_better=False)

for i in range(15):
    number = i
    sheet_name = sheet_names[number]
    print(sheet_name)
    data = pd.read_excel(data_path,skiprows=0,sheet_name=sheet_name )
    data= data.dropna()
    # print(data.columns)

    features=data[data.columns[2:]]
    labels=data[data.columns[1]]

# Standardization
    scaler=preprocessing.StandardScaler().fit(features)
    features_=scaler.transform(features)
    features_=pd.DataFrame(features_,columns=features.columns)
    print(features_.shape)

# Shuffle data order
    indices = np.arange(features_.shape[0])
    np.random.shuffle(indices)
    features_ = features_.iloc[indices]
    labels = labels.iloc[indices]
    
# Define parameter value range
    param_grid = {
    'n_estimators': np.arange(100,200,5),
    # 'max_depth': [8,10,12],
    }
# start prediction
    GBDT = GradientBoostingRegressor()
    scoring = {'R2': 'r2', 'RMSE': rmse}
    
# Create grid search instance
    grid_search = GridSearchCV(estimator=GBDT, param_grid=param_grid, cv=10, scoring=scoring,refit='R2',n_jobs=-1, verbose=2)

# Perform grid search
    grid_search.fit(features_, labels)
    
# Print best parameters and results
    print("Best parameters:", grid_search.best_params_)
    print("Best cross-validation R2 score: {:.4f}".format(grid_search.cv_results_['mean_test_R2'].max()))
    print("Corresponding RMSE for best R2:", grid_search.cv_results_['mean_test_RMSE'][grid_search.cv_results_['mean_test_R2'].argmax()])
    
    # Save result
    best_params = grid_search.best_params_
    best_r2_score = grid_search.cv_results_['mean_test_R2'].max()
    best_rmse_score = grid_search.cv_results_['mean_test_RMSE'][grid_search.cv_results_['mean_test_R2'].argmax()]
    
    data_result = {'name':name,
        # "max_depth": [best_params['max_depth']],
        "n_estimators": [best_params['n_estimators']],
        "Best R2 Score": [best_r2_score],
        "Corresponding RMSE": [best_rmse_score]
    }

    results_df = pd.DataFrame(data_result)
    results_df.to_excel(writer_r,sheet_name=sheet_name)
    print(results_df)

# Save model
    path = r'./GBDT'
    path_model = path + '/' + sheet_name + '.pkl'
    best_model = grid_search.best_estimator_
    try:
        with open(path_model, 'wb') as f:
            pickle.dump(best_model, f)  
        print("模型已成功保存至:", path_model)
    except Exception as e:
        print("保存模型时出错:", e)  

# Retrieve and rank important features
    importance = pd.Series(best_model.feature_importances_, index = features_.columns)
    importance.sort_values().plot(kind='barh')
    sorted_features = sorted(zip(importance,features_.columns),reverse=True)
    sorted_features=pd.DataFrame(sorted_features)
    sorted_features.to_excel(writer_t,index=0,sheet_name=sheet_name)
    print(sorted_features)

In [ ]:
writer_r.save()
writer_t.save()
writer_r.close()
writer_t.close()